In [183]:
!pip install psycopg2-binary sqlalchemy pandas

In [1]:
import psycopg2
import pandas as pd

# Database connection parameters
host = "localhost"  # Use "host.docker.internal" if running Jupyter from another container
port = "26257"      # CockroachDB default port
dbname = "defaultdb"  # Replace with your database name
user = "admin"  # Replace with your user
password = "admin"  # Replace with your password
sslmode = "disable"  # Set to "require" if SSL is enabled

# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port,
        sslmode=sslmode
    )
    print("Connected to CockroachDB successfully!")
except Exception as e:
    print(f"Connection error: {e}")

# Create a cursor to execute queries
cur = conn.cursor()

Connected to CockroachDB successfully!


In [332]:
cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    id SERIAL PRIMARY KEY,
    name STRING,
    age INT
    )
""")
conn.commit()
print("Table created successfully!")

Table created successfully!


In [333]:
cur.execute("INSERT INTO users (name, age) VALUES (%s, %s)", ("Alice", 30))
conn.commit()
print("Data inserted successfully!")

Data inserted successfully!


In [334]:
cur.execute("SELECT * FROM users")
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=["id", "name", "age"])
df


,id,name,age
0,1053191606694117377,Alice,30


In [335]:
cur.execute("UPDATE users SET age = %s WHERE name = %s", (35, "Alice"))
conn.commit()
print("Alice's age updated successfully!")


Alice's age updated successfully!


In [336]:
cur.execute("SELECT * FROM users")
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=["id", "name", "age"])
df

,id,name,age
0,1053191606694117377,Alice,35


In [337]:
cur.execute("ALTER TABLE users ADD COLUMN region STRING DEFAULT 'us-east'")
conn.commit()
print("Region column added!")

Region column added!


In [338]:
cur.execute("SELECT * FROM users")
rows = cur.fetchall()
df = pd.DataFrame(rows)
df

,0,1,2,3
0,1053191606694117377,Alice,35,us-east


In [339]:
cur.execute("SELECT * FROM users")
rows = cur.fetchall()
df = pd.DataFrame(rows)
df

,0,1,2,3
0,1053191606694117377,Alice,35,us-east


In [340]:
cur.execute("DELETE FROM users WHERE name = %s", ("Alice",))
conn.commit()
print("Alice deleted successfully!")


Alice deleted successfully!


In [341]:
cur.execute("SELECT * FROM users")
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=["id", "name", "age","region"])
df

,id,name,age,region


In [342]:
cur.execute("""
CREATE TABLE IF NOT EXISTS logs (
    id SERIAL PRIMARY KEY,
    event STRING,
    user_id INT REFERENCES users(id),
    timestamp TIMESTAMP DEFAULT now()
);
""")
conn.commit()
print("Logs table created!")


Logs table created!


In [344]:
try:
    cur.execute("BEGIN;")
    cur.execute("INSERT INTO users (name, age) VALUES ('Alice', 30) RETURNING id;")
    user_id = cur.fetchone()[0]
    cur.execute("INSERT INTO logs (event, user_id) VALUES ('User Created', %s);", (user_id,))
    cur.execute("COMMIT;")
    print("Transaction committed successfully!")
except Exception as e:
    cur.execute("ROLLBACK;")
    print(f"Transaction failed: {e}")


Transaction committed successfully!


In [345]:
cur.execute("SELECT * FROM logs")
rows = cur.fetchall()
df = pd.DataFrame(rows)
df

,0,1,2,3
0,1053191718092406785,User Created,1053191718076743681,2025-03-09 00:10:05.368569


In [346]:
cur.execute("SELECT * FROM users AS OF SYSTEM TIME '-5s';")  # Read slightly old data
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=["id", "name", "age","region"])
df


,id,name,age,region
0,1053191718076743681,Alice,30,us-east


In [347]:
cur.execute("ALTER TABLE users ADD COLUMN metadata JSONB;")
conn.commit()
print("Metadata column added!")

Metadata column added!


In [348]:
cur.execute("""
INSERT INTO users (name, age, metadata) 
VALUES ('Bob', 28, '{"city": "New York", "hobbies": ["chess", "coding"]}');
""")
conn.commit()
print("Bob inserted successfully!")

Bob inserted successfully!


In [349]:
cur.execute("SELECT name, metadata->>'city' FROM users;")
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=["name", "city"])
df


,name,city
0,Alice,None
1,Bob,New York


In [350]:
cur.execute("SELECT current_user;")
print(cur.fetchone()[0])


admin


In [351]:
cur.execute("SHOW DATABASES;")
rows = cur.fetchall()
print(rows)


[('bigdata_db', 'root', None, None, [], None), ('defaultdb', 'root', None, None, [], None), ('postgres', 'root', None, None, [], None), ('system', 'node', None, None, [], None)]


In [352]:
cur.execute("SHOW TABLES;")
rows = cur.fetchall()
print(rows)


[('public', 'logs', 'table', 'admin', 0, None), ('public', 'users', 'table', 'admin', 1, None)]


In [353]:
cur.execute("SHOW COLUMNS FROM users;")
rows = cur.fetchall()
print(rows)


[('id', 'INT8', False, 'unique_rowid()', '', ['users_pkey'], False), ('name', 'STRING', True, None, '', ['users_pkey'], False), ('age', 'INT8', True, None, '', ['users_pkey'], False), ('region', 'STRING', True, "'us-east'", '', ['users_pkey'], False), ('metadata', 'JSONB', True, None, '', ['users_pkey'], False)]


In [354]:
cur.execute("SELECT * FROM crdb_internal.node_transactions;")
rows = cur.fetchall()
print(rows)


[('8c97ebcd-935c-4837-90fe-1f86c6fe537b', 1, '182af648fa44cb780000000000000001', datetime.datetime(2025, 3, 8, 23, 15, 3, 48943), '"sql txn" meta={id=8c97ebcd key=/Min iso=Serializable pri=0.03663325 epo=0 ts=1741475703.299853264,0 min=1741475703.299853264,0 seq=0} lock=false stat=PENDING rts=1741475703.299853264,0 wto=false gul=1741475703.799853264,0', '', 1, 0, 0, '', 'SERIALIZABLE', 'normal', 'regular'), ('fef3fe69-bf1d-4ae3-a6f4-6481dfbc0207', 1, '182af9e147c81dae0000000000000001', datetime.datetime(2025, 3, 9, 0, 9, 17, 478192), '"sql txn" meta={id=fef3fe69 key=/Min iso=Serializable pri=0.00475058 epo=0 ts=1741478957.486770894,0 min=1741478957.486770894,0 seq=0} lock=false stat=PENDING rts=1741478957.486770894,0 wto=false gul=1741478957.986770894,0', '', 1, 0, 0, '', 'SERIALIZABLE', 'normal', 'regular'), ('f687c0ae-2629-46c9-ae75-e3557a8c9066', 1, '182af9e4af59c4f40000000000000001', datetime.datetime(2025, 3, 9, 0, 10, 37, 17562), '"sql txn" meta={id=f687c0ae key=/Min iso=Serializ

In [355]:
cur.execute("SELECT * FROM crdb_internal.index_usage_statistics;")
rows = cur.fetchall()
print(rows)


[(120, 2, 6, datetime.datetime(2025, 3, 9, 0, 10, 37, 22382, tzinfo=datetime.timezone.utc)), (121, 1, 1, datetime.datetime(2025, 3, 9, 0, 10, 8, 860660, tzinfo=datetime.timezone.utc))]


In [356]:
cur.execute("SELECT COUNT(*) FROM users;")
rows = cur.fetchall()
print("Total users:", rows[0][0])


Total users: 2


In [357]:
cur.execute("SELECT MIN(age), MAX(age) FROM users;")
rows = cur.fetchall()
print("Youngest age:", rows[0][0], "Oldest age:", rows[0][1])


Youngest age: 28 Oldest age: 30


In [358]:
cur.execute("SELECT age, COUNT(*) FROM users GROUP BY age ORDER BY age;")
rows = cur.fetchall()
print(rows)


[(28, 1), (30, 1)]


In [318]:
try:
    cur.execute("BEGIN;")
    cur.execute("INSERT INTO users (name, age) VALUES ('Test User', 99);")
    cur.execute("ROLLBACK;")  # This prevents the data from actually being inserted
    print("Transaction rolled back successfully!")
except Exception as e:
    print(f"Transaction failed: {e}")


Transaction rolled back successfully!


In [320]:
cur.execute("SELECT * FROM crdb_internal.cluster_queries;")
rows = cur.fetchall()
print(rows)



[('182af8593624c9d00000000000000001', '3cf3ecf7-f556-4f56-b773-18e34effa389', 1, '182af7f823e98a540000000000000001', 'admin', datetime.datetime(2025, 3, 8, 23, 41, 7, 976143, tzinfo=datetime.timezone.utc), 'SELECT * FROM crdb_internal.cluster_queries', '172.17.0.1:59308', '', False, 'executing', False, 'AgHa////nxkAAP7/AQAAAAYc', 'defaultdb')]


In [359]:
cur.execute("""
CREATE INDEX IF NOT EXISTS idx_logs_user_id 
ON logs (user_id);
""")
conn.commit()
print("Index on user_id created successfully!")


Index on user_id created successfully!


In [364]:
cur.execute("SELECT * FROM logs WHERE user_id = 1;")
rows = cur.fetchall()
print(rows)


[]


In [365]:
cur.execute("""
CREATE INDEX IF NOT EXISTS idx_logs_timestamp 
ON logs (timestamp DESC);
""")
conn.commit()
print("Index on timestamp created successfully!")


Index on timestamp created successfully!


In [366]:
cur.execute("SELECT * FROM logs WHERE timestamp > now() - INTERVAL '1 day';")
rows = cur.fetchall()
print(rows)


[(1053191718092406785, 'User Created', 1053191718076743681, datetime.datetime(2025, 3, 9, 0, 10, 5, 368569))]


In [2]:
cur.execute("DROP TABLE logs ;")
cur.execute("DROP TABLE users ;")
conn.commit()
print("tables dropped successfully!")


tables dropped successfully!


In [321]:
conn.rollback()
print("Transaction rolled back. You can now execute new commands.")

Transaction rolled back. You can now execute new commands.


In [3]:
cur.close()
conn.close()
print("Connection closed.")

Connection closed.
